In [11]:
# If running in Google Colab, download all of this repo's files
import os
import zipfile
from urllib.request import urlopen
from io import BytesIO

if 'COLAB_GPU' in os.environ:
    repo_url = "https://github.com/Zungrysoft/VoxelGPT/archive/refs/heads/master.zip"
    response = urlopen(repo_url)
    zip_content = response.read()
    with BytesIO(zip_content) as zip_file:
            with zipfile.ZipFile(zip_file) as z:
                z.extractall("/content/")

    !mv -vf /content/VoxelGPT-master/* /content/

In [1]:
# Imports
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import random
from datetime import datetime
import json
import training
import color
import render
import vector as vec

In [2]:
# Constants

# Network Structure
CONTEXT_SIZE = 32       # How many other voxels are considered for a training example
EMBEDDING_SIZE = 520    # Dimensionality of the voxel embedding vector
STACKED_LAYERS = 1      # How many times the network structure repeats itself
ATTENTION_HEADS = 10    # Number of heads in each multi-headed attention mechanism

# Training Hyperparameters
CHECK_RADIUS = 7        # How far away voxels can be to be part of a training example
CENTER_FOCUS = 0.3      # How much to focus on picking voxels close to the center of the cube. Must be between 0 and 1.
LEARNING_RATE = 1e-3
AIR_WEIGHT = 9

# Training data
BATCH_SIZE = 128
TRAINING_EXAMPLES = BATCH_SIZE * 7

In [3]:
# Load voxel palette
# The output is a 255-dimensional vector of probabilities for different colors
# Which 255 colors can be generated is decided by the palette file

# Index 0 is reserved as 'undecided' voxel
# Index 1 is reserved as 'air' voxel
# Index 2-255 are colors. So there are 254 possible colors.
palette = color.load_palette()
palette_size = len(palette)

print(f"Palette has {palette_size} colors")

Palette has 256 colors


In [4]:
# Loss function and optimizer
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

In [5]:
# Create model
def main_model():
    input = keras.Input(shape=(CONTEXT_SIZE, EMBEDDING_SIZE,), name='input')

    # Normalization
    x = keras.layers.LayerNormalization(name=f'normalization_start_a')(input)

    # Stacked layers
    for i in range(STACKED_LAYERS):
        # Multi-headed attention
        fx = keras.layers.MultiHeadAttention(
            num_heads=ATTENTION_HEADS,
            key_dim=EMBEDDING_SIZE,
            name=f'multi_head_attention_{i}',
        )(x, x, use_causal_mask=True)

        # Residual connection
        x = keras.layers.Add(name=f'residual_connection_{i}a')([x,fx])

        # Normalization
        x = keras.layers.LayerNormalization(name=f'normalization_{i}a')(x)

        # Feedforward
        fx = keras.layers.Dense(EMBEDDING_SIZE, name=f'feedforward_{i}')(x)
        fx = keras.layers.LeakyReLU(name=f'relu_{i}')(fx)

        # Residual connection
        x = keras.layers.Add(name=f'residual_connection_{i}b')([x,fx])

        # Normalization
        x = keras.layers.LayerNormalization(name=f'normalization_{i}b')(x)

    # Final feedforward layer
    # Output size should be palette_size-1, since we don't want it to be able to choose "undecided"
    x = keras.layers.Dense(palette_size-1, name='feedforward_final')(x)

    # Softmax
    x = keras.layers.Softmax(name='softmax')(x)
    
    # Build and return model
    return keras.Model(inputs=[input, ], outputs=x)

model = main_model()
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 520)]    0           []                               
                                                                                                  
 normalization_start_a (LayerNo  (None, 32, 520)     1040        ['input[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 multi_head_attention_0 (MultiH  (None, 32, 520)     10832120    ['normalization_start_a[0][0]',  
 eadAttention)                                                    'normalization_start_a[0][0]']  
                                                                                              

In [ ]:
# Design training examples
training_examples = training.generate_training_examples(TRAINING_EXAMPLES, CONTEXT_SIZE)

print(f"{len(training_examples)} training examples created.")

print(random.choice(training_examples))

In [6]:
def encode_training_input(example):
    inputEntry = []

    # Encode context vector
    for index, voxel in enumerate(example):
        if len(inputEntry) < CONTEXT_SIZE+1:
            inputEntry.append(training.embed(index, voxel[0], voxel[1], palette, EMBEDDING_SIZE))

    # Pad remainder of context with zeros
    if len(inputEntry) < CONTEXT_SIZE+1:
        zero_elem = [0,] * EMBEDDING_SIZE
        inputEntry += [zero_elem,] * ((CONTEXT_SIZE+1) - len(inputEntry))

    return inputEntry

def encode_training_output(example):
    outputEntry = []
    for voxel in example:
        outputEntry.append(training.encode_one_hot(voxel[1], palette_size))
    return outputEntry

# Reformat training examples into tensor format
def encode_training_examples(training_examples):
    training_inputs = []
    training_outputs = []
    for example in training_examples:
        input_entry = encode_training_input(example)
        output_entry = encode_training_output(example)
        
        # Shift the output
        input_entry.pop(-1)
        output_entry.pop(0)

        # Push to training example list
        training_inputs.append(input_entry)
        training_outputs.append(output_entry)

    training_input_tensor = tf.Variable(training_inputs, tf.float64)
    training_output_tensor = tf.Variable(training_outputs, tf.float64)

    return training_input_tensor, training_output_tensor

In [ ]:
training_input_tensor, training_output_tensor = encode_training_examples(training_examples)

In [ ]:
# Batch training data
dataset_input = tf.data.Dataset.from_tensor_slices(training_input_tensor)
dataset_output = tf.data.Dataset.from_tensor_slices(training_output_tensor)

training_input_batched = dataset_input.batch(BATCH_SIZE)
training_output_batched = dataset_output.batch(BATCH_SIZE)

In [7]:
def pick_weighted_random(probabilities):
    choice = random.random()
    for i in range(len(probabilities)):
        choice -= probabilities[i]
        if choice < 0:
            return i + 1
    return 1

def pick_highest(probabilities):
    chosen_voxel = 1
    best = 0
    for i in range(len(probabilities)):
        if probabilities[i] > best:
            best = probabilities[i]
            chosen_voxel = i+1
    
    return chosen_voxel

def pick_voxel_from_probabilities(probabilities):
    return pick_weighted_random(probabilities)

# Construct an example sculpture using the model's current progress
# TODO: Move this to a separate file so we can do multithreading and other improvements
def build_sculpture(count, base=None, base_voxels=140, temperature=1.0, debug=False):
    # If a base wasn't specified, create one
    color = int(random.random() * 254) + 1
    # color = 1
    voxels = {}
    start_pos = (0, 0, 0)
    # start_pos = (int(random.random() * training.SIZE[0]), int(random.random() * training.SIZE[1]), int(random.random() * training.SIZE[2]))
    voxels[vec.ttos(start_pos)] = color
    
    # Build context vector for the sculpture
    context = [(start_pos, color)]

    for i in range(count-1):
        # Determine where the next voxel will go
        # TODO: Encode this data into the model somehow
        next_pos = training.pick_next_voxel(voxels, context)

        if i < base_voxels and base != None:
            # Get the voxel from the sculpture
            pos_str = vec.ttos(next_pos)
            if pos_str in base:
                chosen_voxel = base[pos_str]
            else:
                chosen_voxel = 1

            # Build the voxel
            voxels[vec.ttos(next_pos)] = chosen_voxel
            context.append((next_pos, chosen_voxel))
            if len(context) > CONTEXT_SIZE:
                context = training.remove_farthest(context, next_pos)
        else:
            # Get the output from the model
            input_data = encode_training_input(context)
            input_data.pop(-1)
            input_tensor = tf.Variable(input_data, tf.float64)
            input_tensor = tf.reshape(input_tensor, [1, -1, EMBEDDING_SIZE])

            output = model([input_tensor], training=False)
            if debug:
                print(f"Context index: {len(context)-1}")
                print(f"Context at index: {context[len(context)-1]}")
            output_probabilities = output[0][len(context)-1]

            # Pick which voxel to generate based on output probabilities
            # TODO: Implement temperature
            chosen_voxel = pick_voxel_from_probabilities(output_probabilities)

            if debug:
                print(f"Chosen voxel: {float(chosen_voxel)}")
            
            # Build the voxel
            voxels[vec.ttos(next_pos)] = chosen_voxel
            context.append((next_pos, chosen_voxel))
            if len(context) > CONTEXT_SIZE:
                context = training.remove_farthest(context, next_pos)
            
            if debug:
                print()

        
    return voxels

In [ ]:
# Function for saving sculptures to json
def save_sculpture(sculpture, filename):
    json_data = {
        "size": {
            "x": training.SIZE[0],
            "y": training.SIZE[1],
            "z": training.SIZE[2],
        },
        "voxels": sculpture,
    }
    with open(filename, 'w') as output_file:
        json.dump(json_data, output_file, indent=2)

In [ ]:
# Training step
@tf.function
def train_step(input_data, output_data):
    # Set up tape
    with tf.GradientTape() as tape:
      output = model([input_data], training=True)

      loss = loss_function(output_data, output)

      gradients = tape.gradient(loss, model.trainable_variables)

      optimizer.apply_gradients(zip(gradients, model.trainable_variables))

      return loss

# Training loop
def train(epochs):
  time_started = datetime.now()
  example_time = 3
  sculptures_made = 0

  # Epochs
  for epoch in range(epochs):
    # Minibatches
    for (batch_input, batch_output) in zip(training_input_batched, training_output_batched):
      train_step(batch_input, batch_output)

    # Print status
    # print(f"Completed Epoch {epoch}")

    # Check if we should output an example sculpture
    if (datetime.now()-time_started).total_seconds() >= example_time:
      print(f"Epoch {epoch}")
      print("Building example sculpture...")
      sculpture = build_sculpture(training.SIZE[0] * training.SIZE[1] * training.SIZE[2])
      # sculpture = build_sculpture(300)
      sculptures_made += 1
      save_time = (time_started-datetime.utcfromtimestamp(0)).total_seconds()
      sculpture_filename = f"examples/json/example_{save_time}_{sculptures_made}.json"
      save_sculpture(sculpture, sculpture_filename)
      model.save(f"backups/backup_{save_time}_{sculptures_made}.h5")
      render.plot_voxel(sculpture)
      print("Done")
      example_time = (datetime.now()-time_started).total_seconds() * 1.3

train(10000000)

In [ ]:
model.save('trained.h5')

In [ ]:
with open("training/json/juice.json") as base_model_file:
    base_model = json.load(base_model_file)["voxels"]
    for i in range(5):
        sculpture = build_sculpture(training.SIZE[0] * training.SIZE[1] * training.SIZE[2], base=base_model, base_voxels=147)
        save_sculpture(sculpture, f"examples/json/test{i}.json")